# ChatQwQ

这可以帮助您开始使用 QwQ [聊天模型](../../concepts/chat_models.mdx)。有关所有 ChatQwQ 功能和配置的详细文档，请参考[API 参考](https://pypi.org/project/langchain-qwq/)。

## 概览
### 集成详情


| 类 | 包 | 本地 | 可序列化 | 包下载量 | 包最新版本 |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwQ](https://pypi.org/project/langchain-qwq/) | [langchain-qwq](https://pypi.org/project/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### 模型功能
| [工具调用](../../how_to/tool_calling.ipynb) | [结构化输出](../../how_to/structured_output.ipynb) | JSON 模式 | [图像输入](../../how_to/multimodal_inputs.ipynb) | 音频输入 | 视频输入 | [Token 级流式传输](../../how_to/chat_streaming.ipynb) | 原生异步 | [Token 使用量](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |❌  | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ | 

## 设置

要访问 QwQ 模型，您需要创建一个阿里云账户，获取 API 密钥，并安装 `langchain-qwq` 集成包。

### 凭证

前往[阿里云 API 密钥页面](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en)注册阿里云账户并生成 API 密钥。完成此操作后，设置 `DASHSCOPE_API_KEY` 环境变量：

In [2]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### 安装

LangChain QwQ 集成位于 `langchain-qwq` 包中：

In [ ]:
%pip install -qU langchain-qwq

## 实例化

现在我们可以实例化模型对象并生成聊天补全：

In [3]:
from langchain_qwq import ChatQwQ

llm = ChatQwQ(
    model="qwq-plus",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # other params...
)

## 调用

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into French. Let\'s start by breaking down the sentence. The subject is "I", which in French is "Je". The verb is "love", which in this context is present tense, so "aime". The object is "programming". Now, "programming" in French can be "la programmation". \n\nWait, should it be "programmation" or "programmation"? Let me confirm the spelling. Yes, "programmation" is correct. Now, putting it all together: "Je aime la programmation." Hmm, but in French, there\'s a tendency to contract "je" and "aime". Wait, actually, "je" followed by a vowel sound usually takes "j\'". So it should be "J\'aime la programmation." \n\nLet me double-check. "J\'aime" is the correct contraction for "I love". The definite article "la" is needed because "programmation" is a feminine noun. Yes, "programmation" is a feminine noun, so "la" is correct. \n\nIs there any ot

## 链式调用

我们可以像这样将我们的模型与提示模板[链接](../../how_to/sequence.ipynb)起来：

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into German. Let me think. The verb "love" is "lieben" or "mögen" in German, but "lieben" is more like love, while "mögen" is prefer. Since it\'s about programming, which is a strong affection, "lieben" is better. The subject is "I", which is "ich". Then "programming" is "Programmierung" or "Coding". But "Programmierung" is more formal. Alternatively, sometimes people say "ich liebe es zu programmieren" which is "I love to program". Hmm, maybe the direct translation would be "Ich liebe die Programmierung." But maybe the more natural way is "Ich liebe es zu programmieren." Let me check. Both are correct, but the second one might sound more natural in everyday speech. The user might prefer the concise version. Alternatively, maybe "Ich liebe die Programmierung." is better. Wait, the original is "programming" as a noun. So using the noun for

## 工具调用
ChatQwQ 支持工具调用 API，允许你描述工具及其参数，并让模型返回一个 JSON 对象，其中包含要调用的工具和传递给该工具的输入。

### 与 `bind_tools` 一起使用

In [6]:
from langchain_core.tools import tool
from langchain_qwq import ChatQwQ


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwQ()

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s 5 times forty two". Let me break this down. First, I need to identify the numbers involved. The first number is 5, which is straightforward. The second number is forty two, which is 42 in digits. The operation they want is multiplication.\n\nLooking at the tools provided, there\'s a function called multiply that takes two integers. So I should use that. The parameters are first_int and second_int. \n\nI need to convert "forty two" to 42. Since the function requires integers, both numbers should be in integer form. So 5 and 42. \n\nNow, I\'ll structure the tool call. The function name is multiply, and the arguments should be first_int: 5 and second_int: 42. I\'ll make sure the JSON is correctly formatted without any syntax errors. Let me double-check the parameters to ensure they\'re required and of the right type. Yep, both are required and integers. \n\nNo examples were provided, but the function\'s 

## API 参考

有关 ChatQwQ 所有功能和配置的详细文档，请前往 [API 参考](https://pypi.org/project/langchain-qwq/)